In [13]:
import os
import json
import cv2
from tqdm import tqdm

def convert_txt_to_json(image_path, txt_file, json_file, label_dict):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    image = cv2.imread(image_path)
    image_height, image_width = image.shape[:2]

    data = {
        "version": "0.3.3",
        "flags": {},
        "shapes": [],
        "imagePath": image_path,
        "imageData": None,
        "imageHeight": image_height,
        "imageWidth": image_width,
    }

    for line in lines:
        line = line.strip().split()
        txt_label = line[0]
        if txt_label not in label_dict:
            print(f"Warning: Unknown label '{txt_label}' in the txt file '{txt_file}'.")
            continue

        class_id = label_dict[txt_label]
        x_center = float(line[1])
        y_center = float(line[2])
        width = float(line[3])
        height = float(line[4])

        x_min = int((x_center - width / 2) * image_width)
        y_min = int((y_center - height / 2) * image_height)
        x_max = int((x_center + width / 2) * image_width)
        y_max = int((y_center + height / 2) * image_height)

        shape = {
            "label": str(txt_label),
            "text": "",
            "points": [[x_min, y_min], [x_max, y_max]],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        }

        data["shapes"].append(shape)

    with open(json_file, 'w') as f:
        json.dump(data, f)



In [14]:
txt_folder = " "  # 输入存放txt文件的文件夹路径
json_folder = " "  # 输出存放json文件的文件夹路径
image_folder = " "  # 图像文件夹路径
image_files = os.listdir(image_folder)
print(image_files[:3])
if not os.path.exists(json_folder):
    os.makedirs(json_folder)
# 用户提供的标签类别到class_id的字典映射
label_dict = {
    # txt : json
    "0": 1,
    # 添加更多标签类别及其对应的class_id
}

# 批量处理每个txt文件，转换为json格式
for image_file in tqdm(image_files):
    base_name = image_file.split('.')[0]
    txt_file = os.path.join(txt_folder, base_name + ".txt")
    if not os.path.exists(txt_file):
        continue
    image_path = os.path.join(image_folder, image_file)

    json_file = os.path.join(json_folder, base_name + ".json")
    convert_txt_to_json(image_path, txt_file, json_file, label_dict)


['000001_0_0.png', '000001_0_1.png', '000001_0_2.png']


100%|███████████████████████████████████████████████████████████████████████████████| 605/605 [00:02<00:00, 251.93it/s]
